In [ ]:
import numpy as np

class HGWOS:
    def __init__(self, pop=25, iters=40, switch_prob=0.5, seed=42):
        self.pop = pop
        self.iters = iters
        self.switch_prob = switch_prob
        self.rng = np.random.default_rng(seed)

    def _sigmoid(self, x):
        return 1.0 / (1.0 + np.exp(-x))

    def optimize(self, D, fitness_fn):
        # Initialize continuous positions then binarize
        X = self.rng.uniform(-1, 1, size=(self.pop, D))

        best_mask = None
        best_fit = -1e9

        # main loop
        for t in range(self.iters):
            # Evaluate and rank
            fits = np.array([fitness_fn((self._sigmoid(X[i]) > 0.5).astype(np.int32)) for i in range(self.pop)])
            idx = np.argsort(fits)[::-1]  # descending
            alpha, beta, delta = X[idx[0]], X[idx[1]], X[idx[2]]

            if fits[idx[0]] > best_fit:
                best_fit = fits[idx[0]]
                best_mask = (self._sigmoid(alpha) > 0.5).astype(np.int32)

            a = 2 - (2 * t / max(1, self.iters - 1))  # decreases 2 -> 0

            # update each agent
            for i in range(self.pop):
                r = self.rng.random()
                if r < self.switch_prob:
                    # GWO-style update
                    A1 = 2*a*self.rng.random(D) - a
                    C1 = 2*self.rng.random(D)
                    D_alpha = np.abs(C1*alpha - X[i])
                    X1 = alpha - A1*D_alpha

                    A2 = 2*a*self.rng.random(D) - a
                    C2 = 2*self.rng.random(D)
                    D_beta = np.abs(C2*beta - X[i])
                    X2 = beta - A2*D_beta

                    A3 = 2*a*self.rng.random(D) - a
                    C3 = 2*self.rng.random(D)
                    D_delta = np.abs(C3*delta - X[i])
                    X3 = delta - A3*D_delta

                    X[i] = (X1 + X2 + X3) / 3.0
                else:
                    # WOA-style update
                    b = 1.0
                    l = self.rng.uniform(-1, 1, size=D)
                    p = self.rng.random()
                    if p < 0.5:
                        A = 2*a*self.rng.random(D) - a
                        C = 2*self.rng.random(D)
                        Dp = np.abs(C*alpha - X[i])
                        X[i] = alpha - A*Dp
                    else:
                        Dp = np.abs(alpha - X[i])
                        X[i] = Dp * np.exp(b*l) * np.cos(2*np.pi*l) + alpha

        return best_mask, best_fit
